# Some shit with randomization

In [1]:
import random
import os
from random import randint

I used numbers to get 10 families with <=5 sequences

##### Choose the numbers of families

In [29]:
gen_range, numbers, amount_of_fam = int(input('Print number RANGE')), [], int(input('Print size of the sample'))

while amount_of_fam > 0:
    number = random.randint(1, gen_range)
    if number not in numbers:
        numbers.append(number)
        amount_of_fam -= 1

Print number RANGE3115
Print size of the sample10


##### Downloading files

In [37]:
with open("files_to_download.txt", 'w') as out:
    for num in numbers:
        out.write('ftp://ftp.ebi.ac.uk/pub/databases/Rfam/14.1/fasta_files/RF' + ((5 - len(str(num))) * '0' + str(num)) + '.fa.gz' + '\n')
! wget -i "files_to_download.txt" -P "./newfiles/"

##### Unpacking and deleting archives

In [53]:
os.chdir("./newfiles/")
! gunzip *.gz

FileNotFoundError: [Errno 2] No such file or directory: './newfiles/'

##### Here we determine the number of seq form each family (<=5); in case FOR 10 FAMILIES

In [5]:
! grep -c "^>" *.fa >> number_of_seqs.txt
with open("number_of_seqs.txt", 'r') as fname:
    num_of_seq = []
    for num in fname:
        num_of_seq.append(int(num.strip().split(":")[1]))
    need_num = min(num_of_seq) if min(num_of_seq) <= 5 else 5

##### Now we make our sample :)

In [93]:
# need_num
# >RNU6-511P_ENSG00000200366.1,0
import glob
from random import sample
from Bio import SeqIO

out = open("sample.fasta", 'a')
fam_name = 0

for file in glob.glob("./*.fa"):
    fam = file[2:-3]
    record = SeqIO.to_dict(SeqIO.parse(file, "fasta"))
    sp = random.sample(record.keys(), need_num)
    for seq_name in sp:
        out.write(">" + fam + "_" + str(record[seq_name].id).split("/")[0] + "," + str(fam_name) + '\n')
        out.write(str(record[seq_name].seq) + '\n')
    fam_name += 1
    
out.close()

In [94]:
! grep -c "^>" sample.fasta

40


##### Making genelabel file

In [89]:
from Bio import SeqIO
with open("genelabel_sample.txt", 'a') as out:
    for record in SeqIO.parse("sample.fasta", "fasta"):
        iden = str(record.id).split(",")
        out.write(iden[0] + ":" + iden[1] + '\n')

In [90]:
! grep -c ":" genelabel_sample.txt

40
